In [ ]:
# from pandasql import sqldf
from IPython.display import display, HTML,clear_output
import requests
import pandas as pd
import time
# from bs4 import BeautifulSoup

In [ ]:
url ='https://www.nseindia.com/api/quote-derivative?symbol=BANKNIFTY&identifier=OPTIDXBANKNIFTY'

header = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/111.0.0.0 Safari/537.36",
    "Sec-Fetch-User": "?1", "Accept": "*/*", "Sec-Fetch-Site": "none", "Sec-Fetch-Mode": "navigate",
    "Accept-Encoding": "gzip, deflate, br", "Accept-Language": "en-US,en;q=0.9,hi;q=0.8"
    }
session = requests.session()
nse_live = session.get("http://nseindia.com", headers=header)

In [ ]:
def process_data(contract):
  """Processes the given JSON data into a pandas DataFrame.

  Args:
    data: The JSON data to be processed.

  Returns:
    A pandas DataFrame containing the extracted data.
  """

  # Extract relevant data from the JSON structure
  processed_data = []
  for data in contract:
    metadata = data['metadata']
    market_dept = data['marketDeptOrderBook']
    tradeInfo = data['marketDeptOrderBook']['tradeInfo']
    carryOfCost = data['marketDeptOrderBook']['carryOfCost']
    carry = data['marketDeptOrderBook']['carryOfCost']['carry']
    # otherInfo = data['otherInfo']

    processed_data.append({
        # 'instrumentType': metadata['instrumentType'],
        'expiryDate': metadata['expiryDate'],
        'optionType': metadata['optionType'],
        'strikePrice': metadata['strikePrice'],
        # 'identifier': metadata['identifier'],
        # 'openPrice': metadata['openPrice'],
        # 'highPrice': metadata['highPrice'],
        # 'lowPrice': metadata['lowPrice'],
        # 'closePrice': metadata['closePrice'],
        # 'prevClose': metadata['prevClose'],
        # 'lastPrice': metadata['lastPrice'],
        # 'change': metadata['change'],
        # 'pChange': metadata['pChange'],
        # 'numberOfContractsTraded': metadata['numberOfContractsTraded'],
        # 'totalTurnover': metadata['totalTurnover'],
        # 'underlyingValue': data['underlyingValue'],
        'VFQ': data['volumeFreezeQuantity'],
        'TBQ': market_dept['totalBuyQuantity'],
        'TSQ': market_dept['totalSellQuantity'],
        # 'bestBuy':carryOfCost['price']['bestBuy'],
        # 'bestSell':carryOfCost['price']['bestSell'],
        # 'lastPrice':carryOfCost['price']['lastPrice'],
        # 'carry_bestBuy':carry['bestBuy'],
        # 'carry_bestSell':carry['bestSell'],
        # 'carry_lastPrice':carry['lastPrice'],
        'vmap':tradeInfo['vmap'],
        # 'premiumTurnover':tradeInfo['premiumTurnover'],
        'OI':tradeInfo['openInterest'],
        'COI':tradeInfo['changeinOpenInterest'],
        'PCOI':tradeInfo['pchangeinOpenInterest']
        # Add more fields as needed from marketDeptOrderBook
    })

  # Create a pandas DataFrame
  df = pd.DataFrame(processed_data)
  return df

# Example usage:
# Assuming you have the JSON data in a variable named 'data'


In [ ]:
while True:
    clear_output(wait=True)
    request = session.get(url,headers=header).json()['stocks']
    df = process_data(request)
    df1 = df[ (df['strikePrice']>(int(request[0]['underlyingValue'])-500)) & (df['strikePrice']<(int(request[0]['underlyingValue'])+500)) 
           & (df['expiryDate'] =='28-Aug-2024')]

    display(df1.sort_values('strikePrice',ascending=True))
    time.sleep(2)

,expiryDate,optionType,strikePrice,VFQ,TBQ,TSQ,vmap,OI,COI,PCOI
44,28-Aug-2024,Call,50500,901,434820,91545,656.61,55860,778,1.412440
9,28-Aug-2024,Put,50500,901,111150,423345,135.66,145960,31893,27.959883
21,28-Aug-2024,Put,50600,901,80490,188385,163.70,56153,20873,59.163832
56,28-Aug-2024,Call,50600,901,78090,51795,581.46,18045,2633,17.084090
31,28-Aug-2024,Call,50700,901,97755,105765,512.69,33751,7163,26.940725
13,28-Aug-2024,Put,50700,901,76485,234510,196.78,76892,34635,81.962752
12,28-Aug-2024,Call,50800,901,602145,189075,447.87,58090,24590,73.402985
5,28-Aug-2024,Put,50800,901,142935,576765,231.15,119938,54198,82.442957
2,28-Aug-2024,Put,50900,901,465450,495690,273.05,125189,62919,101.042235
3,28-Aug-2024,Call,50900,901,192750,379065,388.88,89935,33428,59.157273
